# GPU 확인

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 460223101166961888
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14415560704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1956348762170138845
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


# 구글 드라이브 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 전처리 (생략)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
BASE_DIR = "/content/drive/MyDrive"

# 학습코드

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dense, Conv2D
from tqdm import tqdm
from tqdm import trange

from numba import jit, cuda, prange

In [ ]:
labelTable = ["cap_and_hat", "outerwear", "tops", "bottoms", "shoes"]

In [ ]:
xDatas = np.load(f'{BASE_DIR}/xDatas32.npy')
yDatas = np.load(f'{BASE_DIR}/yDatas.npy')

In [ ]:
print(len(xDatas))
print(len(yDatas))

30000
30000


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xTrain, xValidation, yTrain, yValidation = train_test_split(xDatas, yDatas, test_size=0.2, stratify=yDatas, random_state=1120)

In [ ]:
imRows = 32
imCols = 32

inputShape = (imRows, imCols, 1)

In [ ]:
# with tf.device("CPU:0"):
xTrain = tf.convert_to_tensor(xTrain, dtype=tf.float32)
xValidation = tf.convert_to_tensor(xValidation, dtype=tf.float32)
yTrain = tf.one_hot(yTrain, depth=len(np.unique(yTrain)))
yValidation = tf.one_hot(yValidation, depth=len(np.unique(yValidation)))

In [ ]:
num_classes = 5   #10개 클래스 기억
model = Sequential()   #순차적으로 레이어를 더해주는 명령

#첫번째 컨볼루션 레이어
#입력 레이어
#컨볼루션 레이어의 컨볼루션 연산과 폴링 연산 설정
model.add(Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding="valid",
                 input_shape=inputShape))   #서로 다른 64개의 필터를 3x3사이즈로 설정하여 연산함. 이때, 최적화함수는 relu(음수는 0으로 반환), 패딩은 사이즈를 작게 두는 valid 옵션 사용
model.add(MaxPooling2D(2))   #폴링필터를 2x2로 설정하여 연산함. MaxPooling2D(2,2)와 같음
model.add(Dropout(0.3))   #30%의 노드를 삭제함

#두번째 컨볼루션 레이어
model.add(Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(MaxPooling2D(2))
model.add(Dropout(0.3))

#세번째 컨볼루션 레이어 
model.add(Conv2D(128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.3))

#첫번째 완전 연결(Fully Connected) 레이어
model.add(Flatten())   #다차원을 1차원으로 변경해주는 명령

#두번째 완전 연결(Fully Connected) 레이어, 
model.add(Dense(128, kernel_initializer='he_normal', activation='relu'))   #노드의 갯수가 256개인 히든레이어

#세번째 완전 연결(Fully Connected) 레이어, 
model.add(Dense(256, kernel_initializer='he_normal', activation='relu'))

#네번째 완전 연결(Fully Connected) 레이어
#출력 레이어
model.add(Dense(num_classes, activation='softmax'))   #출력 레이어의 노드의 갯수는 분류할 10개 클래스와 같아야 함, softmax() 함수는 학습데이터에 얼마나 가까워졌는지 정도를 보여주는 방법임

In [ ]:
model.summary()

In [ ]:
#모델 학습방식 환경설정(최적화함수, 손실함수, 평가지표)
model.compile(optimizer="adam",
              loss="categorical_crossentropy",     
              metrics=["accuracy"])

In [ ]:
#모델 학습실행하는 명령
with tf.device("GPU:0"):
    tracker = model.fit(xTrain, yTrain,
                    batch_size=120,
                    epochs=200,
                    validation_data=(xValidation, yValidation),
                    verbose=1)

Epoch 1/200
200/200 [==============================] - 11s 9ms/step - loss: 30.8304 - accuracy: 0.4745 - val_loss: 1.5728 - val_accuracy: 0.6082
Epoch 2/200
200/200 [==============================] - 2s 8ms/step - loss: 1.1491 - accuracy: 0.5782 - val_loss: 1.4821 - val_accuracy: 0.6118
Epoch 3/200
200/200 [==============================] - 2s 8ms/step - loss: 1.0756 - accuracy: 0.5990 - val_loss: 1.3979 - val_accuracy: 0.6115
Epoch 4/200
200/200 [==============================] - 2s 8ms/step - loss: 1.0588 - accuracy: 0.6063 - val_loss: 1.3226 - val_accuracy: 0.6117
Epoch 5/200
200/200 [==============================] - 2s 8ms/step - loss: 1.0470 - accuracy: 0.6083 - val_loss: 1.2864 - val_accuracy: 0.6117
Epoch 6/200
200/200 [==============================] - 2s 8ms/step - loss: 1.0442 - accuracy: 0.6088 - val_loss: 1.2530 - val_accuracy: 0.6117
Epoch 7/200
200/200 [==============================] - 2s 8ms/step - loss: 1.0387 - accuracy: 0.6092 - val_loss: 1.2239 - val_accuracy: 0.61

In [ ]:
from datetime import datetime
from pytz import timezone

stringTime = datetime.today().astimezone(timezone('asia/Seoul')).strftime("%Y-%m-%d %H-%M-%S")
pathString = f"{BASE_DIR}/output/{stringTime}"
if not os.path.exists(pathString):
  os.makedirs(pathString)

In [ ]:
modelDescription = model.to_json()
with open(f"{pathString}/model.json", "w") as fileModel:
  fileModel.write(modelDescription)
model.save_weights(f"{pathString}/model.h5")

In [ ]:
historyJson = tracker.history
json.dump(historyJson, open(f"{pathString}/tracker.json", "w"))